In [276]:
from binance.client import Client
import pandas as pd
import plotly.graph_objects as go
from pon import api,secret
import numpy as np
import plotly.io as pio
import schedule
import time
from plotly.subplots import make_subplots
import pandas_ta as ta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [277]:
def get_price_data_binance(ticker:str,limit:int,start_str:str, end_str:str,interval = Client.KLINE_INTERVAL_1DAY)->pd.DataFrame:
    client = Client(api, secret)
    df = pd.DataFrame(client.get_historical_klines(symbol=ticker, interval=interval,start_str=start_str, end_str=end_str))
    df.columns=['date','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    return df
    
def get_list_of_imbalances(df:pd.DataFrame)->list:
    up_imbalances = []
    down_imbalances = []
    df['ATR'] = ta.atr(df['high'],df['low'],df['close'],14)
    for i in range(5, len(df)-2):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        #if (df.iloc[i-1]['close'] - df.iloc[i-1]['open'])/3 < (next_candle['low'] - prev_candle['high']):
        if (prev_candle['high'] < next_candle['low']) and df.iloc[i]['ATR']/(next_candle['low'] - prev_candle['high'])<20:
            middle = round((prev_candle['high']+next_candle['low'])/2,2)
            if df.iloc[i+1]['low'] > middle:
                if df.iloc[i+2]['low'] > middle:
                    if (df.iloc[i-1]['close']-next_candle['low']) > (next_candle['low']-middle) or (next_candle['close']-next_candle['low']) > (next_candle['low']-middle):
                        up_imbalances.append([middle, 
                        prev_candle['high'],
                        df.iloc[i-1]['date'],
                        prev_candle['low']])

        #if (df.iloc[i-1]['open'] - df.iloc[i]['open'])/3 < (prev_candle['low'] - next_candle['high']):
        if (prev_candle['low'] > next_candle['high']) and df.iloc[i]['ATR']/(prev_candle['low'] - next_candle['high'])<20:
            middle = round((prev_candle['low']+next_candle['high'])/2,2)
            if df.iloc[i+1]['high'] < middle:
                if df.iloc[i+2]['high'] < middle:
                    if (next_candle['high']-df.iloc[i-1]['close']) > (middle-next_candle['high']) or (next_candle['high']-next_candle['close']) > (middle-next_candle['high']):
                        down_imbalances.append([middle,
                        prev_candle['low'],
                        df.iloc[i-1]['date'],
                        prev_candle['high']])

    return up_imbalances, down_imbalances

def check_within_imbalances(df:pd.DataFrame, up_imbalances:list, down_imbalances:list)->pd.DataFrame:
    df['within_up_imbalance'] = False
    df['within_down_imbalance'] = False
    df['stop'] = 0
    up_imbalancesCOPY = up_imbalances.copy()
    down_imbalancesCOPY = down_imbalances.copy()
    for i in range(2, len(df)):
        for imbalance in up_imbalancesCOPY:
            # if df.iloc[i]['low'] < imbalance[0] and df.iloc[i]['close'] > imbalance[1] and df.iloc[i]['date'] > imbalance[2]:
            if hammer_up(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date'] > imbalance[2]:
                if (df.iloc[i]['low']<imbalance[0] and df.iloc[i]['close']>imbalance[0] and df.iloc[i]['open']>imbalance[0]):
                    df.loc[i, 'within_up_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['low']
                    
            if hammer_up(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date'] > imbalance[2]:
                if (df.iloc[i]['low']<imbalance[1] and df.iloc[i]['close']>imbalance[1] and df.iloc[i]['open']>imbalance[1]):
                    df.loc[i, 'within_up_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['low']

        for imbalance in down_imbalancesCOPY:
            # if df.iloc[i]['high'] > imbalance[0] and df.iloc[i]['close'] < imbalance[1] and df.iloc[i]['date'] > imbalance[2]:
            if falling_star_down(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date']>imbalance[2]:
                if df.iloc[i]['high']>imbalance[0] and df.iloc[i]['close']<imbalance[0] and df.iloc[i]['open']<imbalance[0]:
                    df.loc[i, 'within_down_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['high']
                    
            if falling_star_down(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date']>imbalance[2]:
                if df.iloc[i]['high']>imbalance[1] and df.iloc[i]['close']<imbalance[1] and df.iloc[i]['open']<imbalance[1]:
                    df.loc[i, 'within_down_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['high']

        for imbalance in up_imbalancesCOPY:
            if df.iloc[i-2]['close'] < imbalance[1] and df.iloc[i-2]['date'] > imbalance[2]:
                up_imbalancesCOPY.remove(imbalance)
        
        for imbalance in down_imbalancesCOPY:
            if df.iloc[i-2]['close'] > imbalance[1] and df.iloc[i-2]['date'] > imbalance[2]:
                down_imbalancesCOPY.remove(imbalance)

    return df

def delete_up_imbalance(up_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in up_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2]  and (candle['close'] < imbalance[0] or candle['low'] < imbalance[0]):
                up_imbalances.remove(imbalance)
    return up_imbalances[-3:]

def delete_down_imbalance(down_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in down_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and (candle['close'] > imbalance[0] or candle['high'] > imbalance[0]):
                down_imbalances.remove(imbalance)
    
    return down_imbalances[-3:]

def get_list_of_levels(df:pd.DataFrame)->list:
    levels = []
    for i in range(len(df)):
        levels.append([df.iloc[i]['high'],df.iloc[i]['date']])
        levels.append([df.iloc[i]['low'],df.iloc[i]['date']])
        
    return levels

def add_moving_average(df:pd.DataFrame)->pd.DataFrame:
    
    df['SMA9'] = df['Close'].rolling(9).mean()
    df['SMA9slope'] = df['SMA9'].pct_change()*1000
    
    df['SMA12'] = df['Close'].rolling(12).mean()
    df['SMA12slope'] = df['SMA12'].pct_change()*1000
    
    df['SMA25'] = df['Close'].rolling(25).mean()
    df['SMA25slope'] = df['SMA25'].pct_change()*1000
    
    df['SMA200'] = df['Close'].rolling(200).mean()
    df['SMA200slope'] = df['SMA200'].pct_change()*1000
    df['SMA200sum'] = df['SMA200slope'].rolling(50).sum()
    
    df['crossunder'] = ((df['SMA12'].shift(1) < df['SMA25'].shift(1)) & (df['SMA12'] > df['SMA25'])).astype(int)
    df['crossunder'] = df['crossunder'].apply(lambda x: 1 if x else 0)



    

    # df['up_reversal'] = ((df['SMAslope'].shift(1) < 0) & (df['SMAslope'] > 0))
    # df['down_reversal'] = ((df['SMAslope'].shift(1) > 0) & (df['SMAslope'] < 0)) 

    return df

def add_rsx_HF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSX'] < 30)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSX'] > 70)

    up_reversals = []
    down_reversals = []
    # df['hidden_up_divergence'] = False
    # df['hidden_down_divergence'] = False
    df['classic_up_divergence'] = False
    df['classic_down_divergence'] = False

    for i in range(0, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if up_reversals:
            if df.iloc[i]['date']>up_reversals[-1][2]:
                if df.iloc[i]['low'] < up_reversals[-1][0]:
                    if df.iloc[i]['RSX'] > up_reversals[-1][1]:
                        df.loc[i, 'classic_up_divergence'] = True

        if down_reversals:
            if df.iloc[i]['date']>down_reversals[-1][2]:
                if df.iloc[i]['high'] > down_reversals[-1][0]:
                    if df.iloc[i]['RSX'] < down_reversals[-1][1]:
                        df.loc[i, 'classic_down_divergence'] = True

        # if (up_reversals and down_reversals):
        #     for up_reversal in up_reversals[-5:]:
        #         if (up_reversal[0] < df.iloc[i]['low']) and (up_reversal[1] > df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
        #             df.loc[i, 'hidden_up_divergence'] = True
                
        #     for down_reversal in down_reversals[-5:]:
        #         if (down_reversal[0] > df.iloc[i]['high']) and (down_reversal[1] < df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
        #             df.loc[i, 'hidden_down_divergence'] = True
    return df

def add_rsx_LF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSXslope'].shift(1).rolling(5).sum()<-6)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSXslope'].shift(1).rolling(5).sum()>6)

    up_reversals = []
    down_reversals = []
    df['classic_up_divergence'] = False
    df['classic_down_divergence'] = False

    for i in range(len(df) - 200, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))
                
        if (up_reversals and down_reversals):
            for up_reversal in up_reversals[-1:]:    
                if (up_reversal[0] > df.iloc[i]['low']) and (up_reversal[1] < df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_up_divergence'] = True

            for down_reversal in down_reversals[-1:]:       
                if (down_reversal[0] < df.iloc[i]['high']) and (down_reversal[1] > df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_down_divergence'] = True
                
    return df

def add_rsx(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    return df

def three_candle_up_fractal(left:list,medium:list,right:list):
    #ohlc - 0123
    if left[0] > left[3] and right[0] < right[3] and left[0] > medium[3] and left[0] < right[3] and (left[0]-left[3])*2 > (right[3]-right[0]):
        return True
        
def three_candle_down_fractal(left:list,medium:list,right:list):
    #ohlc - 0123
    if left[0] < left[3] and right[0] > right[3] and left[0] < medium[3] and left[0] > right[3] and (left[3]-left[0])*2 > (right[0]-right[3]):
        return True
    
def hammer_up(current:list):
    #ohlc - 0123
    #При первом множителе 2 винрейт почти такой же, но сделок больше, что увеличивает прибыль
    if current[0]>current[3]:
        if (current[0]-current[3])*3 < (current[1]-current[2]) and (current[1]-current[0])*4 < (current[0]-current[2]):
            return True
    if current[0]<current[3]:
        if (current[3]-current[0])*3 < (current[1]-current[2]) and (current[1]-current[3])*4 < (current[3]-current[2]):
            return True

def falling_star_down(current:list):
    if current[0]>current[3]:
        if (current[0]-current[3])*3 < (current[1]-current[2]) and (current[3]-current[2])*2 < (current[1]-current[3]):
            return True
    
    if current[0]<current[3]:
        if (current[3]-current[0])*3 < (current[1]-current[2]) and (current[0]-current[2])*2 < (current[1]-current[0]):
            return True
    
def down_engulfing(left:list,right:list):
    #ohlc - 0123
    if left[0]<left[3] and right[3]<left[0] and (left[3]-left[0])*2 > (right[0]-right[3]):
        return True
    
def up_engulfing(left:list,right:list):
    #ohlc - 0123
    if left[0]>left[3] and right[3]>left[0] and (left[0]-left[3])*2 > (right[3]-right[0]):
        return True

def backtest(df: pd.DataFrame, buy_orders: list, sell_orders: list,depo:int,risk:int,chart=False):
    # Initialize variables to store statistics
    startdepo = depo
    buy_profit = 0
    buy_loss = 0
    sell_profit = 0
    sell_loss = 0
        
    win_buy_trades = 0
    lose_buy_trades = 0

    win_sell_trades = 0
    lose_sell_trades = 0
    lose = depo*0.01*risk
    depolist = [depo]
    # Iterate through buy orders
    startvalue = 1
    for order in buy_orders:
        open_price = order[0]
        stop_loss = order[1]
        take_profit = order[2]
        open_date = order[3]

        # Iterate through df starting from the open date
        for i in range(startvalue, len(df)):
            if df.iloc[i]['date'] > open_date:
                if df.iloc[i]['high'] >= take_profit:
                    depo += round((take_profit - open_price)/(open_price - stop_loss),2)*lose-(depo*0.01*risk*0.001)
                    win_buy_trades += 1
                    depolist.append(depo)
                    startvalue = i
                    break
                elif df.iloc[i]['low'] <= stop_loss:
                    depo -= 1*lose+(depo*0.01*0.04)
                    lose_buy_trades += 1
                    depolist.append(depo)
                    startvalue = i
                    break
    
    # Iterate through sell orders
    startvalue = 1
    for order in sell_orders:
        open_price = order[0]
        stop_loss = order[1]
        take_profit = order[2]
        open_date = order[3]
        
        # Iterate through df starting from the open date
        for i in range(startvalue, len(df)):
            if df.iloc[i]['date'] > open_date:
                if df.iloc[i]['low'] <= take_profit:
                    depo += round((open_price - take_profit)/(stop_loss - open_price),2)*lose-(depo*0.01*risk*0.001)
                    win_sell_trades += 1
                    depolist.append(depo)
                    startvalue = i
                    break
                elif df.iloc[i]['high'] >= stop_loss:
                    depo -= 1*lose+(depo*0.01*0.04)
                    lose_sell_trades += 1
                    depolist.append(depo)
                    startvalue = i
                    break
    buy_winrate = 0
    sell_winrate = 0
    if len(buy_orders)>0:
        buy_winrate = round((win_buy_trades)/len(buy_orders)*100,2)
    if len(sell_orders)>0:
        sell_winrate = round((win_sell_trades)/len(sell_orders)*100,2)
    total_trades = win_buy_trades + win_sell_trades + lose_buy_trades + lose_sell_trades
    
    peak = depolist[0]
    max_drawdown = 0
    maxdrawdownpeak = 0
    
    for deposit in depolist:
        peak = max(peak, deposit)
        drawdown = round((peak - deposit)/peak*100,2)
        if max_drawdown < drawdown:
            max_drawdown = max(max_drawdown, drawdown)
            maxdrawdownpeak = peak
            
    print(f"""
        Total trades: {total_trades},
        Total win trades: {win_buy_trades + win_sell_trades}
        Total lose trades: {lose_buy_trades + lose_sell_trades}
        Total buy trades: {win_buy_trades + lose_buy_trades}
        Total sell trades: {win_sell_trades + lose_sell_trades}
        Winrate: {round((win_buy_trades+win_sell_trades)/total_trades*100,2)}%
        Winrate buy trades: {buy_winrate}%
        Winrate sell trades: {sell_winrate}%
        Risk: {risk}%
        Start deposit: {startdepo}$
        Deposit: {round(depo,3)}$
        Max Drawdown(%): {max_drawdown}%
        Max Drawdown($): {round(maxdrawdownpeak*0.01*max_drawdown,2)}$"""
    )
    if chart:
        fig = go.Figure(df=[go.Scatter(x=list(range(len(depolist))), y=depolist,mode='lines')])
        fig.show()


In [278]:
# df_higher_tf = get_price_data_binance('BTCUSDT',1000,start_str='1 Jan, 2023', end_str='25 Jan, 2023',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_lower_tf = get_price_data_binance('BTCUSDT',1000,start_str='1 Jan, 2023', end_str='25 Jan, 2023',interval=Client.KLINE_INTERVAL_15MINUTE)
# df_higher_tf2 = get_price_data_binance('BTCUSDT',1000,start_str='18 Dec, 2022', end_str='7 Jan, 2023',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_higher_tf3 = get_price_data_binance('BTCUSDT',1000,start_str='28 Nov, 2022', end_str='18 Dec, 2022',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_higher_tf4 = get_price_data_binance('BTCUSDT',1000,start_str='7 Nov, 2022', end_str='28 Nov, 2022',interval=Client.KLINE_INTERVAL_30MINUTE)
# enddf = pd.read_csv('enddf.csv')
df = get_price_data_binance('BTCUSDT',1000,start_str='1 Feb, 2023 ', end_str='27 Feb, 2023',interval=Client.KLINE_INTERVAL_5MINUTE)
# stoch = ta.stoch(high=df['High'], low=df['Low'], close=df['Close'])
#df_lower_tf = get_price_data_binance('BTCUSDT',1000,interval=Client.KLINE_INTERVAL_5MINUTE)

In [279]:

# fig = go.Figure(data=[go.Candlestick(x=df['date'],
#                                       open=df['open'],
#                                       high=df['high'],
#                                       low=df['low'],
#                                       close=df['close'])])

df = add_rsx(14,df)

buy_orders = []
sell_orders = []

multiplier = 1
counter = 0
for i in range(20, len(df)-15):
    if hammer_up(df.loc[i-1, ['open', 'high', 'low', 'close']]):
        if min(df.loc[i-4:i-2,'low'])>=df.iloc[i-1]['low'] and (df.iloc[i-1]['high']-df.iloc[i-1]['low'])<(df.iloc[i-2]['high']-df.iloc[i-2]['low']):
            buy_orders.append((df.iloc[i]['open'], df.iloc[i-1]['low'],
            round(df.iloc[i]['open'] + (df.iloc[i]['open']-df.iloc[i-1]['low'])*multiplier,2),
            df.iloc[i]['date']))
            # fig = go.Figure(data=[go.Candlestick(x=df['date'][i-19:i+14],
            #                             open=df['open'][i-19:i+14],
            #                             high=df['high'][i-19:i+14],
            #                             low=df['low'][i-19:i+14],
            #                             close=df['close'][i-19:i+14])])
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['open']],mode='markers', marker=dict(size=4, color='white')))
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i-1]['low']],mode='markers', marker=dict(size=4, color='red')))
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[round(df.iloc[i]['open'] + (df.iloc[i]['open']-df.iloc[i-1]['low'])*multiplier,2)],mode='markers', marker=dict(size=4, color='green')))
            # counter+=1
            # pio.write_image(fig, f'screens\\screen№{counter}.jpg')
    
    if falling_star_down(df.loc[i-1, ['open', 'high', 'low', 'close']]):
        if max(df.loc[i-4:i-2,'high'])<=df.iloc[i-1]['high'] and (df.iloc[i-1]['high']-df.iloc[i-1]['low'])<(df.iloc[i-2]['high']-df.iloc[i-2]['low']):
            sell_orders.append((df.iloc[i]['open'], df.iloc[i-1]['high'],
            round(df.iloc[i]['open'] - (df.iloc[i-1]['high']-df.iloc[i]['open'])*multiplier,2),
            df.iloc[i]['date'])) 
            # fig = go.Figure(data=[go.Candlestick(x=df['date'][i-19:i+14],
            #                             open=df['open'][i-19:i+14],
            #                             high=df['high'][i-19:i+14],
            #                             low=df['low'][i-19:i+14],
            #                             close=df['close'][i-19:i+14])])
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['open']],mode='markers', marker=dict(size=4, color='white')))
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i-1]['high']],mode='markers', marker=dict(size=4, color='red')))
            # fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[round(df.iloc[i]['open'] - (df.iloc[i-1]['high']-df.iloc[i]['open'])*multiplier,2)],mode='markers', marker=dict(size=4, color='green')))
            # counter+=1
            # pio.write_image(fig, f'screens\\screen№{counter}.jpg')
                 
print('Переходим к бектесту')

backtest(df,buy_orders,sell_orders,100,1,chart=False)

# for order in buy_orders:
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[0]],mode='markers', marker=dict(size=4, color='white')))
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[1]],mode='markers', marker=dict(size=4, color='red')))
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[2]],mode='markers', marker=dict(size=4, color='green')))

# for order in sell_orders:
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[0]],mode='markers', marker=dict(size=4, color='white')))
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[1]],mode='markers', marker=dict(size=4, color='red')))
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[2]],mode='markers', marker=dict(size=4, color='green')))


# for i in range(3, len(df)):
#     if hammer_up(df.loc[i-1, ['open', 'high', 'low', 'close']]):
#         fig.add_trace(go.Scatter(x=[df.iloc[i-1]['date']],y=[df.iloc[i-1]['low']],mode='markers', marker=dict(size=4, color='green')))
#         # fig.add_vline(x=df_higher_tf.iloc[i-1]['date'],line=dict(color='green'))

#     if falling_star_down(df.loc[i-1, ['open', 'high', 'low', 'close']]):
#         fig.add_trace(go.Scatter(x=[df.iloc[i-1]['date']],y=[df.iloc[i-1]['high']],mode='markers', marker=dict(size=4, color='red')))
#         # fig.add_vline(x=df_higher_tf.iloc[i-1]['date'],line=dict(color='red'))

# for i in range(3, len(df_lower_tf)):
#     if df_lower_tf.iloc[i]['within_up_imbalance']:
#         fig1.add_trace(go.Scatter(x=[df_lower_tf.iloc[i]['date']],y=[df_lower_tf.iloc[i]['stop']],mode='markers', marker=dict(size=4, color='green')))
        
#     if df_lower_tf.iloc[i]['within_down_imbalance']:
#         fig1.add_trace(go.Scatter(x=[df_lower_tf.iloc[i]['date']],y=[df_lower_tf.iloc[i]['stop']],mode='markers', marker=dict(size=4, color='red')))

# atr = go.Figure(go.Scatter(
#         x=df_higher_tf['date'],
#         y=df_higher_tf['ATR']
#     ))



# fig.update_layout(showlegend=False)
# fig.show()
# fig1.update_layout(showlegend=False)
# fig1.show()
# fig2.update_layout(showlegend=False)
# fig2.show()
# atr.update_layout(showlegend=False)
# atr.show()

Переходим к бектесту

        Total trades: 248,
        Total win trades: 145
        Total lose trades: 103
        Total buy trades: 76
        Total sell trades: 172
        Winrate: 58.47%
        Winrate buy trades: 52.63%
        Winrate sell trades: 61.05%
        Risk: 1%
        Start deposit: 100$
        Deposit: 136.94$
        Max Drawdown(%): 12.05%
        Max Drawdown($): 14.03$
